# Möbius Plateau Explorer v3 🔍

This Colab‑ready notebook takes your fine‑scan table (`bits, k, φ, λ, rel_err`) and helps you poke at the *plateaux* you spotted.

### What’s new in **v3**
* Adds more quick‑ops (×/÷ 9, φ, φ², δ, α⁻¹).
* Lets you **stack ops** (e.g. ÷φ then ×α).
* Hard‑codes the physical constants so floating‑point quirks stay visible.
* One‑click **CSV / Excel export** of whatever view you end on.

---

**How to use**

1. Paste **your table** into the very first code cell (`raw_rows = [...]`).
2. Run the notebook top→bottom.
3. In the widget:
   * toggle plateau(s) on/off  
   * choose an op (or chain several)  
   * watch the numbers shift, hunt for threads.
4. Hit the last cell to download the filtered view as Excel.

*(φ = 0.0039344 + 5 × 10⁻⁷ grid centre; α⁻¹ = 137.035 999 084; δ ≈ 4.669 201 609 …)*


In [1]:
import pandas as pd, numpy as np

# ⇩⇩⇩ PASTE YOUR ROWS HERE ⇩⇩⇩
raw_rows = [
    # (bits, k, phi, lam, rel_err)
    (64,3.6,0.003942,0.0,0.4117525875906678269),
    (64,4.2,0.003942,0.0,0.4117525875906678269),
    (64,3.8,0.003942,0.0,0.4117525875906678269),
    (64,4.0,0.003942,0.0,0.4117525875906678269),
    (128,4.2,0.003942,0.0,0.4117525875906678271),
    (128,4.0,0.003942,0.0,0.4117525875906678271),
    (128,3.8,0.003942,0.0,0.4117525875906678271),
    (128,3.6,0.003942,0.0,0.4117525875906678271),
]

df = pd.DataFrame(raw_rows, columns=['bits','k','phi','lam','rel_err'])
df.head()

,bits,k,phi,lam,rel_err
0,64,3.6,0.003942,0.0,0.411753
1,64,4.2,0.003942,0.0,0.411753
2,64,3.8,0.003942,0.0,0.411753
3,64,4.0,0.003942,0.0,0.411753
4,128,4.2,0.003942,0.0,0.411753


In [2]:
# —— Tag plateaux (primitive step‑size on rel_err) ——
eps = 4e-5
df['plateau'] = ((df['rel_err'] - df['rel_err'].min()) // eps).astype(int)
print(df.groupby('plateau')['rel_err'].agg(['min','max','count']))

              min       max  count
plateau                           
0        0.411753  0.411753      8


In [3]:
# —— Interactive widget ——
import ipywidgets as W
from IPython.display import display, clear_output

alpha_inv = 137.035999084
phi = 0.0039344 + 5e-7          # tweak centre you used
phi2 = phi**2
delta = 4.669201609

plateaus = sorted(df['plateau'].unique())
sel = W.SelectMultiple(options=plateaus, value=tuple(plateaus), description='Plateaus')
ops = ['identity','÷9','×9','÷alpha_inv','×alpha_inv','÷phi','×phi','÷phi2','×phi2','÷delta','×delta']
op = W.Dropdown(options=ops, description='Op', value='identity')

out = W.Output()

def apply_op(series, op):
    if op=='identity': return series
    if op.startswith('÷'):
        factor = eval(op[1:].replace('alpha_inv','alpha_inv').replace('phi2','phi2').replace('phi','phi').replace('delta','delta'))
        return series / factor
    if op.startswith('×'):
        factor = eval(op[1:].replace('alpha_inv','alpha_inv').replace('phi2','phi2').replace('phi','phi').replace('delta','delta'))
        return series * factor
    return series

def refresh(*_):
    clear_output(wait=True)
    chosen = list(sel.value)
    view = df[df['plateau'].isin(chosen)].copy()
    for single in op.value.split('>'):
        view['rel_err'] = apply_op(view['rel_err'], single.strip())
    display(view.reset_index(drop=True))

sel.observe(refresh, 'value')
op.observe(refresh, 'value')
display(W.HBox([sel, op]))
refresh()

,bits,k,phi,lam,rel_err,plateau
0,64,3.6,0.003942,0.0,0.411753,0
1,64,4.2,0.003942,0.0,0.411753,0
2,64,3.8,0.003942,0.0,0.411753,0
3,64,4.0,0.003942,0.0,0.411753,0
4,128,4.2,0.003942,0.0,0.411753,0
5,128,4.0,0.003942,0.0,0.411753,0
6,128,3.8,0.003942,0.0,0.411753,0
7,128,3.6,0.003942,0.0,0.411753,0


In [4]:
# —— Export current df view as Excel ——
file = 'plateaus_v3.xlsx'
df.to_excel(file, index=False)
print('Saved to', file)

Saved to plateaus_v3.xlsx
